In [ ]:
pip install dnspython

# Join collection

## Importo librerie necessarie

In [ ]:
from pymongo import MongoClient
import time as time
from tqdm.notebook import tqdm

# Merge pagina singola

In [ ]:
# Connect mongodb atlas
from pymongo import MongoClient
access_token = ""
client = MongoClient(access_token)
db = client.get_database('wikitrend')
pages = db.pages_full

In [ ]:
start = time.perf_counter()
# lookup pipline
result = db.page_topic.aggregate([
    {
        '$lookup': {
            'from': 'info_pages', 
            'localField': '_id', 
            'foreignField': '_id', 
            'as': 'page_info'
        }
    }
])
result = list(result)
# clean document
for articolo in result:
  page_info = articolo['page_info'][0]
  del page_info["article"]
  del page_info["_id"]
  del articolo['page_info']
  articolo.update(page_info)
# upload document
pages.insert_many(result)
fin = time.perf_counter() - start
print(fin)

# Merge top pages

In [ ]:
# Connect mongodb atlas
from pymongo import MongoClient
access_token = ""
client = MongoClient(access_token)
db = client.get_database('wikitrend')
top_pages = db.top_pages.find()
top_pages_full = db.top_pages_full
page_topic = db.page_topic

In [ ]:
top_pages = list(top_pages)

In [ ]:
start = time.perf_counter()
for doc in tqdm(top_pages):
  # extract array
  pagine_mese = doc["top_month"]
  pagine_giorno = doc["top_day"]
  # create auxilary collections
  mese_coll = db.mese_coll
  day_coll = db.dey_coll
  mese_coll.insert_many(pagine_mese)
  day_coll.insert_many(pagine_giorno)
  # join top_month with page_topic
  join_mese = mese_coll.aggregate([
    {
        '$lookup': {
            'from': 'page_topic', 
            'localField': 'page_id', 
            'foreignField': '_id', 
            'as': 'cat_list'
        }
    }
  ])
  join_mese = list(join_mese)
  # join top_day with page_topic
  join_giorno = day_coll.aggregate([
    {
        '$lookup': {
            'from': 'page_topic', 
            'localField': 'page_id', 
            'foreignField': '_id', 
            'as': 'cat_list'
        }
    }
  ])
  join_giorno = list(join_giorno)  
  # delete auxialary collections
  mese_coll.drop()
  day_coll.drop()
  # extract main attribute
  for articolo in join_mese:
    try:
      articolo["category"] = articolo["cat_list"][0]["category"]
    except:
      pass
    try:
      articolo["page_type"] = articolo["cat_list"][0]["page_type"]
    except:
      pass
    try:
      articolo["description"] = articolo["cat_list"][0]["description"]
    except:
      pass
    del articolo["cat_list"]
    del articolo["_id"]
  for articolo in join_giorno:
    try:
      articolo["category"] = articolo["cat_list"][0]["category"]
    except:
      pass
    try:
      articolo["page_type"] = articolo["cat_list"][0]["page_type"]
    except:
      pass
    try:
      articolo["description"] = articolo["cat_list"][0]["description"]
    except:
      pass
    del articolo["cat_list"]
    del articolo["_id"]
  doc["top_month"] = join_mese
  doc["top_day"] = join_giorno
  # upload doc
  top_pages_full.insert_one(doc)
fin = time.perf_counter() - start
print(fin)
